## Task 1：引言

### 1.序列到序列（Seq2Seq）模型概述

#### 1.1 什么是Seq2Seq模型

**SeqSeq模型**：输入一个序列，输出一个序列。（输入、输出序列的长度*可变*）

# ![seq2seq模型示意图](image/1.1.png)

*可变*值得就是图中的输入了5个汉字，输出了4个英文单词。

```
<bos>：Begin Of Sequence（BOS）
<eos>：End Of Sequence（EOS）
```
> 使用BOS和EOS标记句子的开始和结束，可以减少对**填充（padding）**的依赖，模型可根据这些标记识别序列的边界。

#### 1.2 SeqSeq的由来

**Padding可能引入的问题**：在某些任务中，将文本表示成序列后，其长度事先并不知道，无法进行padding。

基于此，Seq2Seq框架应运而生。

#### 1.3 Seq2Seq模型的评价

##### 1.3.1 优势：

1. 端到端学习Seq2Seq：实现了原始输入数据（如文本、语音等）到期望输出序列的直接映射。*无需显性的特征提取或工程化步骤*
2. 处理可变长度序列：可适应不同长度的序列数据。
3. 信息压缩与表示：
    1. 编码器通过编码将输入序列压缩为一个固定维度上下文向量，可作为输入序列的抽象表示；
    2. 解码器基于次上下文向量进行解码生成目标输出序列。
4. 可拓展性Seq2Seq：具有良好的模块化特性，可与CNNs、RNNs等神经网络架构无缝集成。

##### 1.3.2 缺点：

1. 上下文向量信息压缩：输入序列被压缩成一个固定维度的上下文向量，会导致信息压缩和损失问题。（细粒度细节丢失）
2. 短期记忆限制：Seq2Seq的编码器和解码器都是基于RNN，Seq2Seq模型在处理长序列时存在短期记忆限制，难以有效捕获和传递长期依赖性。限制模型在处理具有长距离时间步依赖的序列时的性能。
3. 暴露偏差（Exposure Bias）：模型在训练过程中采用的是“teacher forcing”策略，在每个时间步提供真实的输出作为解码器的输入。与模型在推理的自回归生成模式存在不一致性，模型在测试的时候无法适应其自身的错误输出。

> **Teacher Forcing**：核心思想是在训练过程中，使用*真实的目标输出*（即训练数据中的正确答案）作为模型每一步的输入，而不是使用模型自己在前一时刻预测的输出。在推理阶段，由于没有真实的目标输出可用，模型必须依赖自己之前的预测结果来生成下一个词。这时模型的表现可能会与训练时有所不同。<br>优点：加速收敛、减少错误积累。<br>缺点：训练与推理不一致（Exposure Bias）。<br>解决方法：使用Scheduled Sampling和强化学习等。


#### 2.1 "Seq2Seq"和”Encoder-Decoder“的关系

Seq2Seq模型**强调目的**。是Encoder-Decoder架构的具体应用。

Encoder-Decoder模型**强调方法**。

**Encoder-Decoder**：将现实问题转化为数学问题，通过求解数学问题去解决现实问题。无论输入和输出的长度是什么，中间的“上下文向量”长度都是固定的。

#### 2.2 Encoder-Decoder的工作流程

##### 2.2.1 编码器（Encoder）

编码器是一个RNN，由多个循环单元（如LSTM或GRU）堆叠而成。每个单元通过依次处理输入序列中的一个元素，生成一个汇总了输入序列全部信息的上下文向量。这个向量在所有单元间传递当前元素的信息，构建对整个序列的理解。

**工作流程**：

1. **词序列转换**：**词嵌入（word embedding）**将每个词汇映射到一个*高维空间中的稠密向量*，这些向量携带了词汇的语义信息。
2. **序列处理**：嵌入的向量序列经过基于RNN的结构处理，捕捉输入序列中的顺序信息和上下文关系。
3. **生成上下文向量**：RNN的*最后一个隐藏层输出*被用作整个输入序列的表示，称为“**上下文向量（context vector）**。是一个*固定长度*的向量。

##### 2.2.2 解码器（Decoder）

解码器接收编码器输出的上下文向量作为初始输入，并依次合成目标序列的各个元素。

**工作流程**：

1. **初始化参数**：初始化解码器RNN及其相关输出层的参数。参数包括：
    1. 权重（Weights）：连接不同神经网络层的参数，决定了层与层之间之间信息的传递方式。
    2. 偏置（Biases）：神经网络层中的固定值，用于调节激活函数的输出。

> 常用的初始化策略：<br>**Xavier初始化（Glorot初始化）**：适用于*Sigmoid或Tanh激活函数*，核心思想是让权重的初始值在一个合适的范围内，从一个均匀分布（范围是基于$ \pm \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}} $的边界）或正态分布（标准差是基于$ \sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}} $）中随机采样，使得每一层的输入和输出的方差保持一致。<br>**He初始化（Kaiming初始化）**：适用于*ReLU激活函数及其变体*，因为ReLU激活函数的特性会把所有负值都截断，导致信号在网络中的传播变得不对称。从一个均匀分布（范围是基于$\pm \sqrt{\frac{6}{n_{\text{in}}}}$的边界）或正态分布（标准差是基于$ \sqrt{\frac{2}{n_{\text{in}}}} $）中随机采样。（*He初始化只考虑了输入单元数$n_{in}$，没有同时考虑$n_{in}+n_{out}$。因为ReLU会将负值截断为零，导致输出的方差主要取决于输入的方差。

2. **编码器输出**：负责对源序列通过RNN进行初步处理：
    1. 输入序列处理：对序列中的每一个元素按序处理；
    2. 隐藏状态更新：根据输入元素和当前状态通过不同RNN动态更新隐藏状态。
    3. 上下文向量生成：编码器的最后一个隐藏状态被用作上下文向量。

3. **解码器输入**：
    1. 初始隐藏状态：上下文向量通常被设定为解码器的初始隐藏状态。或者与解码器的第一个隐藏状态合并（拼接）。
    2. 开始符号（bos：Begin Of Sequence）：解码器由开始符号作为触发信号来启动序列的生成。
    3. 输入序列（Input Sequence）：第一个时间步之后，解码器的输入将是前一个时间步的输出。
    4. 上下文向量：每个时间步，解码器会接收到一个上下文向量。
    5. 注意力权重（Attention Weights）：如果模型采用注意力机制，那每个时间步解码器还会接收一组注意力权重。这些权重决定编码器输出中哪些部分对当前时间步的解码最重要。

##### 2.2.3 Seq2Seq模型的训练过程

1. **准备数据**：
    * 数据预处理：分词+编码（增加bos和eos）；
    * 批量处理
2. **初始化模型参数**：
    * 解码器RNN：初始化解码器的权重和偏置。
    * 输出层：初始化输出层的权重和偏置，通常是一个全连接层。将解码器的隐藏状态映射到目标词汇的概率分布。
3. **编码器处理**：
    * 运行编码器：通过编码器RNN处理源序列生成上下文向量。
4. **解码器训练过程**：
    * 初始化隐藏状态：使用编码器的最后一个隐藏状态或上下文向量来初始化解码器的隐藏状态。
    * 时间步迭代：
        * 输入：根据开头标记开始复述。（Teacher Forcing）
        * 解码器RNN：根据当前时间步的输入和前一个时间步的隐藏状态，计算当前时间步的隐藏状态。
        * 输出层：当前时间步的隐藏状态通过输出层，得到一个概率分布，表示当前时间步预测的目标词汇。
        * 损失计算：计算预测的概率分布和实际目标词汇之间的损失（**交叉熵损失**）
5. **反向传播和参数更新**：
    * 计算梯度：对损失函数进行反向传播，计算模型参数的梯度。
    * 参数更新：根据计算出的梯度更新解码器的权重和偏置。
6. **循环训练**：
    * 多个epoch：多次训练直到模型收敛。
7. **评估和调优**：
    * 验证集评估：定期在验证集上测试模型预测能力。
    * 超参数调优：根据验证结果，调整**学习率、epoch**等超参数。

# ![Encoder-Decoder示意图](image/2.2.4.png)

#### 2.3 Encoder-Decoder的应用

# ![Encoder-Decoder应用示意图](image/2.3.png)

* **文本-文本**：机器翻译、对话机器人、诗词生成、代码补全、文章摘要。《Sequence to Sequence Learning with Neural Networks》
* **音频-文本**：语音识别。《A Comparison of Sequence-to-Sequence Models for Speech Recognition》
* **图片-文本**：图像描述生成。Sequence to Sequence – Video to Text》

#### 2.4 Encoder-Decoder的缺陷

**当输入信息太长时，会丢失掉一些信息。**

#### 3.1 Attention的发展历程

# ![Attention的发展历程示意图](image/3.1.png)

#### 3.2 Attention的N种类型

# ![Attention的N种类型示意图](image/3.2.png)

#### 3.3 Attention解决信息丢失问题

# ![引入Attention的Encoder-Decoder示意图](image/3.3.png)

Attention模型的特点是Encoder不再将整个输入序列编码为固定长度的“向量C”，而是编码成一个向量（Context vector）的序列（“C1”、“C2”、“C3”），解决“信息过长，信息丢失”的问题。

#### 3.4 Attention的核心工作

核心工作：**关注重点**。

* 场景：信息量大，包含有效信息和噪声；
* 问题：算力和资源有限，无法同事处理所有信息。
* 应对：快速从复杂信息中检索出对解决问题最重要的信息。

#### 3.5 Attention的2大优点

1. **参数少**：模型复杂度相对于CNN、RNN，复杂度更小，参数更少。
2. **速度快**：解决了RNN不能并行计算的问题。
3. **效果好**：解决了长距离的信息会被弱化问题。



